In [ ]:
# 1번. AlexNet모델구조

In [ ]:
## TODO ##

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_1 = (11*11*3) * 96 + 96 

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Max_pool_1 = 0

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_2 = (5*5*96)*256+256

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Max_pool_2 = 0

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_3 = (3*3*256)*384 +384

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_4 = (3*3*384)*384 +384

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Conv_5 = (3*3*384)*256 +256

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

Max_pool_3 = 0

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

FC1 = (6*6*256)*4096 + 4096

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

FC2 = 4096 * 4096 + 4096

In [ ]:
## TODO 

######################################################
#  Calculate the number of parameters in this layer  #
######################################################

FC3 = 4096 * 1000 + 4096

In [ ]:
# 2번. AlexNet구현

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Lambda
from keras.regularizers import l2
from keras import backend as K
from keras import optimizers
from keras.engine.topology import Layer

In [ ]:
input_shape = (227, 227, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(96, (11, 11), strides=4,
                 padding='same', input_shape=input_shape))
model.add(Conv2D(256, (5, 5), activation='relu', padding='same'))
model.add(LocalResponseNormalization(input_shape=model.output_shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(LocalResponseNormalization(input_shape=model.output_shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

In [ ]:
optimizer = optimizers.SGD(lr=0.01, decay=5e-5, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])
model.summary()

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets,transforms
import torch.optim as optim
from torchsummary import summary
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
training_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transformer)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transformer)

In [ ]:
training_loader = torch.utils.data.DataLoader(dataset=training_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Dropout(0.6),
            nn.Linear(4096, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(0.6),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes),
        )

    def forward(self, x):
        conv_features = self.features(x)
        flatten = conv_features.view(conv_features.size(0), -1)
        fc = self.fc_layers(flatten)
        return fc

In [ ]:
epochs = 20
batch_size = 128
log_batch = 200
train_list = []
test_list = []

In [ ]:
model = AlexNet()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# model summary
summary(model,(3,32,32))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import time

for epoch in range(epochs):
        running_loss = 0.0
        correct_classified = 0
        total = 0
        start_time = time.time()
        for i, data in enumerate(training_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct_classified += (predicted == labels).sum().item()
            running_loss += loss.item()
            if i % log_batch == log_batch - 1:
                avg_loss = running_loss / log_batch
                print('Epoch: %d/%d Batch: %5d loss: %.3f' % (epoch + 1, epochs, i + 1, avg_loss))
                running_loss = 0.0
        print("Time/epoch: {} sec".format(time.time() - start_time))
        train_acc = (100 * correct_classified / total)
        train_list.append(train_acc)
        print('Train accuracy : %d %%' % train_acc)
        torch.save(net.state_dict(), "model.h5") # 모델을 저장
        correct_classified = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data
                inputs, labels = images.to(device), labels.to(device)
                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct_classified += (predicted == labels).sum().item()
            test_acc = (100 * correct_classified / total)
            test_list.append(test_acc)
            print('Test accuracy : %d %%' % test_acc)

In [ ]:
# colab에서 돌린 결과값
# Epoch: 1/20 Batch:   200 loss: 1.271
# Epoch: 1/20 Batch:   400 loss: 1.176
# Time/epoch: 51.72108054161072 sec
# Train accuracy of the network images: 56 %
# Test accuracy of the network: 58 %
# Epoch: 2/20 Batch:   200 loss: 1.048
# Epoch: 2/20 Batch:   400 loss: 1.004
# Time/epoch: 51.59315204620361 sec
# Train accuracy of the network images: 64 %
# Test accuracy of the network: 64 %
# Epoch: 3/20 Batch:   200 loss: 0.903
# Epoch: 3/20 Batch:   400 loss: 0.888
# Time/epoch: 51.604798316955566 sec
# Train accuracy of the network images: 68 %
# Test accuracy of the network: 68 %
# Epoch: 4/20 Batch:   200 loss: 0.780
# Epoch: 4/20 Batch:   400 loss: 0.809
# Time/epoch: 51.63756847381592 sec
# Train accuracy of the network images: 72 %
# Test accuracy of the network: 70 %
# Epoch: 5/20 Batch:   200 loss: 0.712
# Epoch: 5/20 Batch:   400 loss: 0.720
# Time/epoch: 51.69989371299744 sec
# Train accuracy of the network images: 75 %
# Test accuracy of the network: 71 %
# Epoch: 6/20 Batch:   200 loss: 0.637
# Epoch: 6/20 Batch:   400 loss: 0.666
# Time/epoch: 51.617228746414185 sec
# Train accuracy of the network images: 77 %
# Test accuracy of the network: 72 %
# Epoch: 7/20 Batch:   200 loss: 0.592
# Epoch: 7/20 Batch:   400 loss: 0.610
# Time/epoch: 51.64090609550476 sec
# Train accuracy of the network images: 79 %
# Test accuracy of the network: 74 %
# Epoch: 8/20 Batch:   200 loss: 0.547
# Epoch: 8/20 Batch:   400 loss: 0.561
# Time/epoch: 51.69233179092407 sec
# Train accuracy of the network images: 80 %
# Test accuracy of the network: 75 %
# Epoch: 9/20 Batch:   200 loss: 0.486
# Epoch: 9/20 Batch:   400 loss: 0.519
# Time/epoch: 51.62518811225891 sec
# Train accuracy of the network images: 82 %
# Test accuracy of the network: 74 %
# Epoch: 10/20 Batch:   200 loss: 0.458
# Epoch: 10/20 Batch:   400 loss: 0.494
# Time/epoch: 51.65696334838867 sec
# Train accuracy of the network images: 83 %
# Test accuracy of the network: 75 %
# Epoch: 11/20 Batch:   200 loss: 0.413
# Epoch: 11/20 Batch:   400 loss: 0.458
# Time/epoch: 51.61623287200928 sec
# Train accuracy of the network images: 84 %
# Test accuracy of the network: 75 %
# Epoch: 12/20 Batch:   200 loss: 0.407
# Epoch: 12/20 Batch:   400 loss: 0.427
# Time/epoch: 51.5536003112793 sec
# Train accuracy of the network images: 85 %
# Test accuracy of the network: 75 %
# Epoch: 13/20 Batch:   200 loss: 0.370
# Epoch: 13/20 Batch:   400 loss: 0.391
# Time/epoch: 51.65504288673401 sec
# Train accuracy of the network images: 86 %
# Test accuracy of the network: 75 %
# Epoch: 14/20 Batch:   200 loss: 0.353
# Epoch: 14/20 Batch:   400 loss: 0.377
# Time/epoch: 51.59739279747009 sec
# Train accuracy of the network images: 87 %
# Test accuracy of the network: 76 %
# Epoch: 15/20 Batch:   200 loss: 0.324
# Epoch: 15/20 Batch:   400 loss: 0.352
# Time/epoch: 51.5103600025177 sec
# Train accuracy of the network images: 88 %
# Test accuracy of the network: 76 %
# Epoch: 16/20 Batch:   200 loss: 0.292
# Epoch: 16/20 Batch:   400 loss: 0.348
# Time/epoch: 51.63554072380066 sec
# Train accuracy of the network images: 88 %
# Test accuracy of the network: 75 %
# Epoch: 17/20 Batch:   200 loss: 0.287
# Epoch: 17/20 Batch:   400 loss: 0.321
# Time/epoch: 51.56595015525818 sec
# Train accuracy of the network images: 89 %
# Test accuracy of the network: 76 %
# Epoch: 18/20 Batch:   200 loss: 0.273
# Epoch: 18/20 Batch:   400 loss: 0.293
# Time/epoch: 51.497596979141235 sec
# Train accuracy of the network images: 90 %
# Test accuracy of the network: 76 %
# Epoch: 19/20 Batch:   200 loss: 0.252
# Epoch: 19/20 Batch:   400 loss: 0.285
# Time/epoch: 51.631884813308716 sec
# Train accuracy of the network images: 90 %
# Test accuracy of the network: 76 %
# Epoch: 20/20 Batch:   200 loss: 0.245
# Epoch: 20/20 Batch:   400 loss: 0.265
# Time/epoch: 51.55575180053711 sec
# Train accuracy of the network images: 91 %
# Test accuracy of the network: 75 %

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_list, label='Train ACC')
plt.plot(test_list, label='Test ACC')
plt.legend(frameon=False)
plt.show()

: 그래프로 표현했을 때 Train Accuracy는 계속 증가하지만 Test Accuracy는 75%에서 멈춘것을 봐서 overfitting 되어서 75%에서 증가하지않을 것을 알 수 있다.